## Create CONSTANTS

In [46]:
import sys

In [47]:
EPM = {
    'min_util': -sys.float_info.max,
    'min_per': 1,
    'max_per': 6,
    'min_avg': 1,
    'max_avg': 5,
    'top_k' : 10
}

### Show global variable

In [48]:
print(EPM)

{'min_util': -1.7976931348623157e+308, 'min_per': 1, 'max_per': 6, 'min_avg': 1, 'max_avg': 5, 'top_k': 10}


# Reading database

In [49]:
UNIT_UTILITY = {}

DATASET = list()

UNIT_COUNT = {}

with open('data/retail_negative.txt', 'r') as file:
    count = 0
    for line in file:
        temp_struc = line.split(":")
        temp_items = list(map(lambda x: "item" + x, temp_struc[0].split(" ")))
        temp_profits = list(map(int, temp_struc[2].split(" ")))
        temp_transaction = {
            "Tid": "T" + str(count),
            "Item": temp_items,
            "Quantity": [1] * len(temp_profits)
        }

        for i in range(len(temp_items)):
            if temp_items[i] not in UNIT_UTILITY:
                UNIT_UTILITY[temp_items[i]] = 0
                UNIT_COUNT[temp_items[i]] = 0
            UNIT_UTILITY[temp_items[i]] += temp_profits[i]
            UNIT_COUNT[temp_items[i]] += 1
            
        DATASET.append(temp_transaction)
        count += 1
        if count >= 10:
            break
    print(count)
for i in UNIT_UTILITY:
    UNIT_UTILITY[i] /= UNIT_COUNT[i]

LEN_DATASET = len(DATASET)

10


In [50]:
print(len(UNIT_UTILITY))

70


## Preparation Procedure

The Transaction Weight Utility (TWU) of item \( i \) is given by:

Def 6
Def 4
\[
{TWU}(i) = \sum_{{X \in T_j \\ T_j \subseteq D}} P_U(T_j)
\]

calculate maxPer(i), avgPer(i)
Def 1, Def 2

rearrange order
Def 7


In [51]:
# Function to create a periodic high-utility itemset based on given constraints
# Input explain: dataset: dictionary contain all original transactions, unit_utility: utility of each item, epm: contain minUtility, maxper, minper, maxavg, minavg
# Output: temp_dataset: dataset contains transactions with qualified items, EUCS_LIST: contain EUCS of item pairs, order_list: order of items acording to defined order
def create_periodic_high_util_itemset(dataset: list, unit_utility: dict, epm: dict) -> list:
    # Dictionaries to store total weighted utility (TWU) values and transaction-based TWU
    twu_dict = dict()
    twu_transaction_dict = dict()
    
    # Dictionaries for periodic support (PS), transaction records, and filtering
    ps_dict = dict()
    transaction_dict = dict()
    
    # Step 1: Calculate TWU for each item and record transactions
    for i in dataset:
        twu_transaction_dict[i['Tid']] = 0  # Initialize TWU for the transaction
        
        for j in range(len(i['Item'])):
            # Initialize dictionaries for new items
            if i['Item'][j] not in twu_dict:
                twu_dict[i['Item'][j]] = 0
                transaction_dict[i['Item'][j]] = []
                ps_dict[i['Item'][j]] = []

            # Calculate transaction-level TWU
            twu_transaction_dict[i['Tid']] += i['Quantity'][j] * unit_utility[i['Item'][j]] if unit_utility[i['Item'][j]] > 0 else 0
            transaction_dict[i['Item'][j]].append(int(i['Tid'][1:]))  # Store transaction ID
        
        # Aggregate TWU for each item
        for j in range(len(i['Item'])):
            twu_dict[i['Item'][j]] += twu_transaction_dict[i['Tid']]

    # Step 2: Update the minimum utility threshold (min_util) based on top-k selection
    sorted_twu_list = list(sorted(twu_dict.items(), key=lambda item: item[1]))
    epm["min_util"] = sorted_twu_list[epm["top_k"] - 1][1] if epm["top_k"] - 1 < len(sorted_twu_list) else sorted_twu_list[-1][1]

    # Step 3: Compute periodic support (PS) values for each item
    for i in transaction_dict.keys():
        ps_dict[i].append(transaction_dict[i][0] + 1)  # First transaction gap
        
        for j in range(1, len(transaction_dict[i])):
            ps_dict[i].append(transaction_dict[i][j] - transaction_dict[i][j - 1])  # Compute gaps between transactions
        
        # Compute the last gap to the end of the dataset
        if LEN_DATASET - 1 - transaction_dict[i][-1] > 0:
            ps_dict[i].append(LEN_DATASET - 1 - transaction_dict[i][-1])
    
    # Step 4: Filter items based on periodic constraints
    temp_dataset = list()
    eucs_dict = dict()
    order_set = set()

    for i in dataset:
        temp_item_quantity = list()  # Temporary list to store filtered items

        for j in range(len(i['Item'])):
            avg_item = sum(ps_dict[i['Item'][j]]) / len(ps_dict[i['Item'][j]])  # Compute average period
            # Apply filtering conditions based on min_util, max_per, and min_avg
            if twu_dict[i['Item'][j]] > epm['min_util'] and max(ps_dict[i['Item'][j]]) <= epm['max_per'] \
                    and avg_item >= epm['min_avg']:
                
                # Store item details
                temp_item_quantity.append((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))
                order_set.add((i['Item'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))

        # Sort items based on TWU values and unit utility
        temp_item_quantity.sort(key=lambda x: x[3])
        print(temp_item_quantity)  # Debugging print statement
        temp_item_quantity.sort(key=lambda x: x[2])
        print(temp_item_quantity)  # Debugging print statement
        
        # Store filtered transaction data
        temp_dataset.append(
            {
                'Tid': i['Tid'],
                'Item': list(map(lambda x: x[0], temp_item_quantity)),  # Extract sorted item list
                'Quantity': list(map(lambda x: x[1], temp_item_quantity))  # Extract corresponding quantities
            }
        )

    # Step 5: Generate ordered item list for further processing
    order_list = list(order_set)
    order_list.sort(key=lambda x: x[2])  # Sort based on TWU values
    order_list.sort(key=lambda x: x[1])  # Sort based on unit utility
    order_list = [i[0] for i in order_list]  # Extract ordered item names

    # Step 6: Compute pairwise Extended Utility Contribution Score (EUCS)
    for i in temp_dataset:
        for j in range(len(i['Item']) - 1):
            for k in range(j + 1, len(i['Item'])):
                # Create pairwise item combinations and store their TWU contributions
                if i['Item'][j] + i['Item'][k] not in eucs_dict:
                    eucs_dict[i['Item'][j] + i['Item'][k]] = 0
                eucs_dict[i['Item'][j] + i['Item'][k]] += twu_transaction_dict[i['Tid']]

    return temp_dataset, eucs_dict, order_list  # Return processed dataset, EUCS dictionary, and ordered items

In [52]:
NEW_DATASET, EUCS_LIST, ORDER_LIST = create_periodic_high_util_itemset(dataset=DATASET, unit_utility=UNIT_UTILITY, epm=EPM)

[]
[]
[('item32', 1, 29.333333333333332, 106.99999999999999)]
[('item32', 1, 29.333333333333332, 106.99999999999999)]
[]
[]
[('item36', 1, 48.0, 163.33333333333334), ('item37', 1, -15.0, 163.33333333333334), ('item40', 1, -9.0, 163.33333333333334), ('item42', 1, 21.0, 163.33333333333334), ('item43', 1, 3.0, 163.33333333333334), ('item44', 1, 10.0, 163.33333333333334), ('item45', 1, 35.0, 163.33333333333334), ('item46', 1, -27.0, 163.33333333333334), ('item41', 1, -16.0, 200.66666666666669), ('item38', 1, 37.333333333333336, 385.0), ('item39', 1, 9.0, 394.0)]
[('item46', 1, -27.0, 163.33333333333334), ('item41', 1, -16.0, 200.66666666666669), ('item37', 1, -15.0, 163.33333333333334), ('item40', 1, -9.0, 163.33333333333334), ('item43', 1, 3.0, 163.33333333333334), ('item39', 1, 9.0, 394.0), ('item44', 1, 10.0, 163.33333333333334), ('item42', 1, 21.0, 163.33333333333334), ('item45', 1, 35.0, 163.33333333333334), ('item38', 1, 37.333333333333336, 385.0), ('item36', 1, 48.0, 163.33333333333

In [53]:
# Reset EPM
EPM = {
    'min_util': -sys.float_info.max,
    'min_per': 1,
    'max_per': 6,
    'min_avg': 1,
    'max_avg': 5,
    'top_k' : 10
}

In [54]:
for i in NEW_DATASET:
    print(i)

{'Tid': 'T0', 'Item': [], 'Quantity': []}
{'Tid': 'T1', 'Item': ['item32'], 'Quantity': [1]}
{'Tid': 'T2', 'Item': [], 'Quantity': []}
{'Tid': 'T3', 'Item': ['item46', 'item41', 'item37', 'item40', 'item43', 'item39', 'item44', 'item42', 'item45', 'item38', 'item36'], 'Quantity': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'Tid': 'T4', 'Item': ['item48', 'item39', 'item47', 'item38'], 'Quantity': [1, 1, 1, 1]}
{'Tid': 'T5', 'Item': ['item48', 'item49', 'item50', 'item53', 'item52', 'item56', 'item39', 'item51', 'item57', 'item55', 'item58', 'item54', 'item38'], 'Quantity': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'Tid': 'T6', 'Item': ['item41', 'item32'], 'Quantity': [1, 1]}
{'Tid': 'T7', 'Item': ['item48', 'item39'], 'Quantity': [1, 1]}
{'Tid': 'T8', 'Item': [], 'Quantity': []}
{'Tid': 'T9', 'Item': ['item32'], 'Quantity': [1]}


In [55]:
print(EUCS_LIST)

{'item46item41': 163.33333333333334, 'item46item37': 163.33333333333334, 'item46item40': 163.33333333333334, 'item46item43': 163.33333333333334, 'item46item39': 163.33333333333334, 'item46item44': 163.33333333333334, 'item46item42': 163.33333333333334, 'item46item45': 163.33333333333334, 'item46item38': 163.33333333333334, 'item46item36': 163.33333333333334, 'item41item37': 163.33333333333334, 'item41item40': 163.33333333333334, 'item41item43': 163.33333333333334, 'item41item39': 163.33333333333334, 'item41item44': 163.33333333333334, 'item41item42': 163.33333333333334, 'item41item45': 163.33333333333334, 'item41item38': 163.33333333333334, 'item41item36': 163.33333333333334, 'item37item40': 163.33333333333334, 'item37item43': 163.33333333333334, 'item37item39': 163.33333333333334, 'item37item44': 163.33333333333334, 'item37item42': 163.33333333333334, 'item37item45': 163.33333333333334, 'item37item38': 163.33333333333334, 'item37item36': 163.33333333333334, 'item40item43': 163.3333333

## Searching stage

In [56]:
# Function to construct a new itemset by merging two input lists based on utility and periodic conditions
# Input:
#   - prefix_list: List containing prefix utilities (can be None)
#   - x: List containing utility and periodic values for itemset X
#   - y: List containing utility and periodic values for itemset Y
# Output:
#   - result: List containing merged itemsets with updated utility and periodic values
def construct(prefix_list: list, x: list, y: list) -> list:
    pre_pointer = 0  # Pointer for prefix list
    x_pointer = 0  # Pointer for itemset X
    y_pointer = 0  # Pointer for itemset Y
    result = list()  # List to store constructed itemsets

    while x_pointer < len(x) and y_pointer < len(y):
        if x[x_pointer][0] < y[y_pointer][0]:
            x_pointer += 1
        elif x[x_pointer][0] > y[y_pointer][0]:
            y_pointer += 1
        else:
            if prefix_list is not None:
                while prefix_list[pre_pointer][0] < x[x_pointer][0]:
                    pre_pointer += 1
                result.append([
                    pre_pointer,  # Position index in prefix list
                    x[x_pointer][1] + y[y_pointer][1] - prefix_list[pre_pointer][1],  # Adjusted utility
                    min(x[x_pointer][2], y[y_pointer][2])  # Minimum periodic value
                ])
                x_pointer += 1
                y_pointer += 1
                pre_pointer += 1
            else:
                result.append([
                    x_pointer,
                    x[x_pointer][1] + y[y_pointer][1],  # Sum of utilities
                    min(x[x_pointer][2], y[y_pointer][2])  # Minimum periodic value
                ])
                x_pointer += 1
                y_pointer += 1
    return result

# Function to check if an itemset meets the minimum utility condition
# Input: 
#   - x: List containing item utilities
#   - minutil: Minimum required utility threshold
# Output:
#   - Boolean: True if total utility of x meets minutil, otherwise False
def utility_condition(x, minutil):
    return sum(i[1] for i in x) >= minutil

# Function to calculate the total utility of an itemset
# Input: 
#   - x: List containing utility values
# Output:
#   - Sum of utility values in x
def get_utility(x):
    return sum(i[1] for i in x)

# Function to calculate periodic statistics of an itemset (max, min, avg)
# Input: 
#   - x: List of item indices
# Output:
#   - max_period: Maximum periodic gap
#   - min_period: Minimum periodic gap (if zero, set to 1)
#   - avg_period: Average periodic gap
def calculate_periodic(x):
    periodic = [x[0][0] + 1]  # Initial periodic gap based on first element
    keys = x  # Store the input list

    for i in range(len(keys) - 1):
        periodic.append(keys[i + 1][0] - keys[i][0])

    periodic.append(LEN_DATASET - 1 - keys[-1][0])  # Last gap calculation

    return max(periodic), min(periodic) if min(periodic) > 0 else 1, sum(periodic) / len(periodic)

# Function to check if an itemset satisfies periodic constraints
# Input:
#   - x: List containing item periodic values
#   - minper, maxper, minavg, maxavg: Constraints on periodic values
# Output:
#   - Boolean: True if itemset meets all constraints, otherwise False
def periodic_condition(x, minper, maxper, minavg, maxavg):
    max_i, min_i, avg_i = calculate_periodic(x)
    return min_i >= minper and max_i <= maxper and avg_i >= minavg and avg_i <= maxavg

# Function to check if an itemset meets the remaining utility (RU) condition
# Input:
#   - x: List containing remaining utility values
#   - minutil: Minimum required utility threshold
# Output:
#   - Boolean: True if remaining utility meets minutil, otherwise False
def ru_condition(x, minutil):
    return sum(i[2] for i in x) >= minutil

# Function to check if an itemset satisfies the maximum periodic condition
# Input:
#   - x: List containing periodic values
#   - maxper: Maximum allowed periodic value
# Output:
#   - Boolean: True if max periodic gap ≤ maxper, otherwise False
def max_per_condition(x, maxper):
    max_x, min_x, avg_x = calculate_periodic(x)
    return max_x <= maxper

# Function to check if an itemset satisfies the average periodic condition
# Input:
#   - x: List containing periodic values
#   - maxavg: Maximum allowed average periodic value
# Output:
#   - Boolean: True if average periodic gap ≤ maxavg, otherwise False
def avg_per_condition(x, maxavg):
    max_x, min_x, avg_x = calculate_periodic(x)
    return avg_x <= maxavg

# Function to generate a combined name for two items
# Input:
#   - X: Name of the first item
#   - Y: Name of the second item
# Output:
#   - String: Concatenated name of the two items
def get_name(X, Y):
    temp_x = X.split("item")  # Extract item number from X
    temp_y = Y.split("item")  # Extract item number from Y
    return "item" + temp_x[-1] + "item" + temp_y[-1]  # Combine item numbers

# Function to check if the extended utility contribution pruning (EUCP) condition is met
# Input:
#   - X, Y: Names of two items
#   - eucs_list: Dictionary containing EUC values of item pairs
#   - minutil: Minimum required utility threshold
# Output:
#   - Boolean: True if EUC value meets minutil, otherwise False
def eucp_condition(X, Y, eucs_list, minutil):
    items_name = get_name(X, Y)  # Generate itemset name
    return eucs_list[items_name] >= minutil


In [57]:
# Function to perform PHM search for periodic high-utility itemsets
# This function applies utility and periodic conditions to identify high-utility itemsets 
# and recursively expands qualified itemsets.
#
# Input:
#   - prefix: Current prefix itemset (used for tracking the sequence)
#   - prefix_list: List containing utility values of the prefix itemset
#   - lists: Dictionary mapping itemsets to their utility and periodic values
#   - order_list: List defining the order of itemsets to be processed
#
# Output:
#   - result: A set of high-utility itemsets that satisfy periodic constraints
def phm_seach(prefix: str, prefix_list: list, lists: dict, order_list: list) -> set:
    result = list()  # Store qualified itemsets
    key_lists = list()  # Filtered order list containing only items present in 'lists'

    # Step 1: Filter items that exist in 'lists'
    for i in order_list:
        if i in lists:
            key_lists.append(i)

    # Step 2: Evaluate each item in the filtered list
    for i in key_lists:
        # Check if the current item satisfies both utility and periodic conditions
        if utility_condition(lists[i], EPM['min_util']) and periodic_condition(lists[i], EPM['min_per'], EPM['max_per'], EPM['min_avg'], EPM['max_avg']):
            result.append((i, get_utility(lists[i])))

            # Maintain only top-k high-utility itemsets
            if len(result) > EPM["top_k"]:
                result.sort(key=lambda x: x[1], reverse=True)  # Sort by utility in descending order
                result = result[:EPM["top_k"]]  # Keep only the top-k elements
                EPM["min_util"] = result[-1][1]  # Update minimum utility threshold

        # Step 3: Check if item is expandable based on remaining utility and periodic conditions
        if ru_condition(lists[i], EPM['min_util']) and max_per_condition(lists[i], EPM['max_per']) and avg_per_condition(lists[i], EPM['max_avg']):
            new_list = dict()  # Dictionary to store new extended itemsets
            new_order_list = list()  # List of new itemsets to be processed

            # Iterate over the remaining items in the list
            for j in key_lists[key_lists.index(i) + 1:]:
                if get_name(i, j) not in EUCS_LIST:
                    continue  # Skip if item pair does not exist in EUCS list

                # Check if the item pair satisfies the EUCP condition
                if eucp_condition(i, j, EUCS_LIST, EPM['min_util']):
                    # Generate the new itemset name
                    z = get_name(i, j) if prefix is None else prefix + get_name(i, j)

                    # Construct a new itemset by merging current items
                    z_list = construct(prefix_list, lists[i], lists[j])

                    # Store the new itemset and update order list
                    new_list[z] = z_list
                    new_order_list.append(z)

            # Recursively search for new periodic high-utility itemsets
            result.extend(phm_seach(i, lists[i], new_list, new_order_list))

            # Maintain only top-k high-utility itemsets after recursion
            if len(result) > EPM["top_k"]:
                result.sort(key=lambda x: x[1], reverse=True)  # Sort by utility in descending order
                result = result[:EPM["top_k"]]  # Keep only the top-k elements
                EPM["min_util"] = result[-1][1]  # Update minimum utility threshold

    return result  # Return the final list of high-utility itemsets


In [58]:
utility_list = dict()
pnu_utility_list = dict()
for i in NEW_DATASET:
    total = 0
    for j in range(len(i['Item']) - 1, -1, -1):

        if i['Item'][j] not in utility_list:
            utility_list[i['Item'][j]] = list()
            pnu_utility_list[i['Item'][j]] = list()

        utility_list[i['Item'][j]].append([int(i['Tid'][1:]), UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], total])
        
        if UNIT_UTILITY[i['Item'][j]] < 0:
            pnu_utility_list[i['Item'][j]].append([int(i['Tid'][1:]), 0, UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], total])
        else:
            pnu_utility_list[i['Item'][j]].append([int(i['Tid'][1:]), UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], 0, total])

        total += UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j] if UNIT_UTILITY[i['Item'][j]] > 0 else 0

In [59]:
result = phm_seach(None, None, utility_list, ORDER_LIST)

In [60]:
print(result)

[('item38', 112.0), ('item32', 88.0), ('item36', 48.0), ('item39', 36.0), ('item45', 35.0), ('item54', 35.0), ('item55', 21.0), ('item58', 21.0), ('item42', 21.0), ('item47', 12.0)]
